In [139]:
#Importation des packages et des fonctions personnalisées
from DataManager import DataManager
from models import GARCHModel, NNARModel, ss_kf_fit, run_nnar_pipeline, run_garch_pipeline
from stats import *
from plotting import *

import pandas as pd
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import multi_dot

import matplotlib.pyplot as plt

import scipy.optimize as sco
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from sklearn.metrics import mean_absolute_error

# Implémentation d'un modèle GarchX 

In [140]:

# Loi normal sur les residus fit mieux que student car pour frequence horaire mouvement extreme moins fréquent.
def LLgarchX(theta, r, x, return_sigma=False):
    
    T = len(r)
    e = r - np.mean(r)  # Centrer les rendements
    e = np.insert(e, 0, 0)  # Première erreur inconditionnelle fixée à 0

    sigma0 = np.var(r)  # Variance inconditionnelle initiale
    alpha1, beta1, gamma = theta  # Paramètres pour GARCH-X
    w = (1 - alpha1 - beta1) * sigma0

    ll = np.zeros(T + 1)
    sigma = np.zeros(T + 1)
    sigma[0] = sigma0

    for t in range(1, T + 1):
        sigma[t] = max(w + alpha1 * e[t - 1]**2 + beta1 * sigma[t - 1] + gamma * x[t - 1]**2, 1e-8)
        z_t = e[t] / np.sqrt(sigma[t])  # Résidus standardisés
        ll[t] = (
            -0.5 * np.log(2 * np.pi)
            - 0.5 * np.log(sigma[t])
            - 0.5 * (z_t**2)
        )
        
    if return_sigma:
        return -np.sum(ll[1:]), sigma[1:]  # Retourne log-vraisemblance et volatilités conditionnelles
    else:
        return -np.sum(ll[1:])

class GarchX:
    """
    Implémentation d'un modèle GARCHX(1,1) avec distribution Normale
    utilisant une fonction de log-vraisemblance personnalisée.
    """
    def __init__(self, scale_factor): #scale factor si jamais on utilise des rendements centrés (c'est pas le cas)
        self.name = 'GARCH(1,1)'
        self.params = None  # Paramètres estimés
        self.fitted_volatility = None  # Volatilité conditionnelle ajustée
        self.scale_factor = scale_factor  # Facteur pour déstandardiser la volatilité 

    def fit(self, y, x):
        """
        Ajuste le modèle GARCHX(1,1) avec la log-vraisemblance 
        y : rendements 
        x : variable exogène 
        """
        # Initialisation des paramètres : alpha1, beta1, gamma, nu
        theta_init = np.array([0.15, 0.8, 0.05])

        # Optimisation de la log-vraisemblance
        result = minimize(
            lambda theta: LLgarchX(theta, y, x),
            theta_init,
            method='L-BFGS-B',
            bounds=[(1e-8, 1), (1e-8, 1), (1e-8, None)],
            options = {'maxiter': 800, 'disp': True}
        )
        if not result.success:
            raise ValueError(f"L'optimisation a échoué : {result.message}")

        # Stockage des résultats
        self.params = result.x

        # Calcul de la volatilité conditionnelle ajustée
        _, self.fitted_volatility = LLgarchX(
            self.params, y, x, return_sigma=True
        )
    
    def fitted_values(self):
        """
        Retourne la volatilité conditionnelle (in-sample) ajustée.
        """
    
        return np.sqrt(self.fitted_volatility) * self.scale_factor  #self.scale_factor utilisé pour déstandardiser si utilisation de rendements centrés

    def calculate_model_errors(self, y_true):
        """
        Calcule les erreurs entre la volatilité conditionnelle ajustée et une volatilité observée.
        y_true : volatilité observée 
        """
        fitted_vol = self.fitted_values()
        residuals = y_true - fitted_vol
        mae = np.mean(np.abs(residuals))
        rmse = np.sqrt(np.mean(residuals**2))
        return {
            'MAE': mae,
            'RMSE': rmse
        }

def run_garch_x(y_returns, y_obs_vol, x_exog, scale_factor=1):
    """
    - Instancie la classe.
    - Fit sur y_returns 
    - Compare à y_obs_vol in-sample.
    - Retourne un dict avec les résultats : fitted, RMSE, MAE
    """
    model = GarchX(scale_factor)
    model.fit(y_returns, x_exog)

    # Volatilité in-sample
    fitted_vol = model.fitted_values()
    # Erreurs in-sample
    errors = model.calculate_model_errors(y_obs_vol)
    
    return {
    'Fitted_values': fitted_vol,
    'MAE': errors['MAE'],
    'RMSE': errors['RMSE'],
    'Params': model.params
}



# Implémentation d'un modèle GARCH in Mean  

In [141]:

def LLgarch_inMean(theta, r, x, return_sigma=False):
    T = len(r)
    sigma0 = np.var(r)  # Variance inconditionnelle pour initialisé
    w = (1 - theta[1] - theta[2]) * sigma0

    # Initialisation des séries
    u = np.zeros(T)  # Résidus conditionnels
    sigma = np.zeros(T)  # Variance conditionnelle
    ll = np.zeros(T)  # Log-vraisemblance
    sigma[0] = sigma0  # Initialisation de la variance conditionnelle

    # Calcul des résidus conditionnels
    for t in range(1, T):
        # Volatilité conditionnelle
        sigma[t] = max(w + theta[1] * u[t - 1]**2 + theta[2] * sigma[t - 1], 1e-8)

        # Moyenne conditionnelle avec x (volatilité du Nasdaq à t-1)
        mean_t = theta[0] + theta[3] * x[t - 1]  # m + δ * x_{t-1}

        # Calcul des résidus
        u[t] = r[t] - mean_t  # u_t = r_t - mean_t

        # Log-vraisemblance
        z_t = u[t] / np.sqrt(sigma[t])  # Standardisation des résidus
        ll[t] = -0.5 * (np.log(2 * np.pi) + np.log(sigma[t]) + z_t**2)

    if return_sigma:
        return -np.sum(ll[1:]), sigma  # Retourne log-vraisemblance et sigma si demandé
    else:
        return -np.sum(ll[1:])  # Retourne uniquement la log-vraisemblance


class GarchinMean:
    """
    Implémentation d'un modèle GARCH in Mean avec intégration de la volatilité du Nasdaq dans l'équation du rendement conditionnel
    utilisant une fonction de log-vraisemblance personnalisée.
    """
    def __init__(self, scale_factor): #scale factor si jamais on utilise des rendements centrés (c'est pas le cas)
        self.name = 'GARCH(1,1)'
        self.params = None  # Paramètres estimés
        self.fitted_volatility = None  # Volatilité conditionnelle ajustée
        self.scale_factor = scale_factor  # Facteur pour déstandardiser la volatilité 

    def fit(self, y, x):
        """
        Ajuste le modèle
        y : rendements 
        x : variable exogène 
        """
        # Initialisation des paramètres : mu, delta, alpha1, beta1
        theta_init = np.array([0.0005, 0.1, 0.15, 0.8])

        # Optimisation de la log-vraisemblance
        result = minimize(
            lambda theta: LLgarch_inMean(theta, y, x),
            theta_init,
            method='L-BFGS-B',
            #bounds=[(None, None), (None, None), (1e-8, 1),(1e-8, 1)],
            options = {'maxiter': 500, 'disp': True}
        )

        if not result.success:
            raise ValueError(f"L'optimisation a échoué : {result.message}")

        # Stockage des résultats
        self.params = result.x

        # Calcul de la volatilité conditionnelle ajustée
        _, self.fitted_volatility = LLgarch_inMean(
            self.params, y, x, return_sigma=True
        )

    def fitted_values(self):
        """
        Retourne la volatilité conditionnelle (in-sample) ajustée.
        """
    
        return np.sqrt(self.fitted_volatility) * self.scale_factor  #self.scale_factor utilisé pour déstandardiser si utilisation de rendements centrés

    def calculate_model_errors(self, y_true):
        """
        Calcule les erreurs entre la volatilité conditionnelle ajustée et une volatilité observée.
        y_true : volatilité observée 
        """
        fitted_vol = self.fitted_values()
        residuals = y_true - fitted_vol
        mae = np.mean(np.abs(residuals))
        rmse = np.sqrt(np.mean(residuals**2))
        return {
            'MAE': mae,
            'RMSE': rmse
        }

def run_garch_inmean(y_returns, y_obs_vol, x_exog, scale_factor=1):
    """
    - Instancie la classe.
    - Fit sur y_returns 
    - Compare à y_obs_vol in-sample.
    - Retourne un dict avec les résultats : fitted, RMSE, MAE
    """
    model = GarchinMean(scale_factor)
    model.fit(y_returns, x_exog)

    # Volatilité in-sample
    fitted_vol = model.fitted_values()
    # Erreurs in-sample
    errors = model.calculate_model_errors(y_obs_vol)
    
    return {
    'Fitted_values': fitted_vol,
    'MAE': errors['MAE'],
    'RMSE': errors['RMSE'],
    'Params': model.params
    #'Shape': model.shape
}



# Téléchargemement des données et affichage des résultats

In [142]:
#chargement des données pour chacune des cryptos ci-dessous :
crypto_currency = ['BTC','ETH','LTC','XRP','BCH']

data = {name: pd.read_excel('data_crypto_vix.xlsx',sheet_name= name, index_col=0, parse_dates=True).dropna() for name in crypto_currency}

#Stockage des datas pour chaque crypto
crypto_returns = {name: data[name]['Log_price']for name in crypto_currency}
obs_vol_crypto = {name: data[name]['obs_volatility'] for name in crypto_currency}
delta_vix = {name: data[name]['Log_Vix']for name in crypto_currency}
obs_vol_nasdaq = {name: data[name]['obs_vol_nasdaq']for name in crypto_currency}

# Standardisation des log returns des cryptos et du VIX
crypto_stdz_returns = {name: (data[name]['Log_price'] - data[name]['Log_price'].mean()) / data[name]['Log_price'].std() 
                  for name in crypto_currency}

delta_stdz_vix = {name: (data[name]['Log_Vix'] - data[name]['Log_Vix'].mean()) / data[name]['Log_Vix'].std() 
             for name in crypto_currency}
 

In [158]:
# Initialisation des résultats
results = []

# Boucle sur chaque crypto pour exécuter les modèles
for name in crypto_currency:
    # Données pour chaque crypto
    y_returns = crypto_returns[name]
    y_obs_vol = obs_vol_crypto[name]
    x_exog = delta_vix[name]
    
    # Modèle 1 : GARCH In-Mean
    result_garch_inmean = run_garch_inmean(y_returns, y_obs_vol, x_exog, scale_factor=1)
    mae_inmean = result_garch_inmean['MAE']
    rmse_inmean = result_garch_inmean['RMSE']
    
    # Modèle 2 : GARCH-X
    result_garchx = run_garch_x(y_returns, y_obs_vol, x_exog, scale_factor=1)
    mae_garchx = result_garchx['MAE']
    rmse_garchx = result_garchx['RMSE']
   
    
    # Modèle 3 : Garch(1,1) classique
    result_pipeline = run_garch_pipeline(
        y_returns=y_returns, 
        y_obs_vol=y_obs_vol,
        horizon=5
    )
    mae_pipeline = result_pipeline['MAE']
    rmse_pipeline = result_pipeline['RMSE']
    
    # Stockage des résultats pour chaque crypto
    results.append({
        'Crypto': name,
        'Model': 'GARCH In-Mean',
        'MAE': mae_inmean,
        'RMSE': rmse_inmean
    })
    
    results.append({
        'Crypto': name,
        'Model': 'GARCH-X',
        'MAE': mae_garchx,
        'RMSE': rmse_garchx
    })
    
    results.append({
        'Crypto': name,
        'Model': 'Garch(1,1)',
        'MAE': mae_pipeline,
        'RMSE': rmse_pipeline
    })

# Création d'un DataFrame pour afficher les résultats
results_df = pd.DataFrame(results).sort_values(by='Model')
results_df



C:\Users\tmeye\AppData\Local\Temp\ipykernel_10836\951453775.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_t = theta[0] + theta[3] * x[t - 1]  # m + δ * x_{t-1}
C:\Users\tmeye\AppData\Local\Temp\ipykernel_10836\951453775.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  u[t] = r[t] - mean_t  # u_t = r_t - mean_t
C:\Users\tmeye\AppData\Local\Temp\ipykernel_10836\951453775.py:15: RuntimeWarning: overflow encountered in scalar multiply
  sigma[t] = max(w + theta[1] * u[t - 1]**2 + theta[2] * sigma[t - 1], 1e-8)
c:\Users\tmeye\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\optimize\_numdiff

,Crypto,Model,MAE,RMSE
0,BTC,GARCH In-Mean,0.004310,0.006038
3,ETH,GARCH In-Mean,0.006556,0.008267
6,LTC,GARCH In-Mean,0.006293,0.007924
9,XRP,GARCH In-Mean,0.009328,0.014936
12,BCH,GARCH In-Mean,0.006269,0.008591
1,BTC,GARCH-X,0.004754,0.007267
4,ETH,GARCH-X,0.008387,0.014251
7,LTC,GARCH-X,0.007001,0.008729
10,XRP,GARCH-X,0.010515,0.016645
13,BCH,GARCH-X,0.006984,0.009272
